# Exercises

1. How would you describe TensorFlow in a short sentence? What are its main features? Can you name other popular deep learning libraries?
2. Is TensorFlow a drop-in replacement for NumPy? What are the main differences between the two?
3. Do you get the same result with `tf.range(10)` & `tf.constant(np.arange(10))`?
4. Can you name six other data structures available in TensorFlow, beyond regular tensors?
5. A custom loss function can be defined by writing a function or by subclassing the `keras.losses.Loss` class. When would you use each option?
6. Similarly, a custom metric can be defined in a function or a subclass of `keras.metrics.Metric`. When would you use each option?
7. When should you create a custom layer versus a custom model?
8. What are some use cases that require writing your own custom training loop?
9. Can custom keras components contain arbitrary python code, or must they be convertible to tf functions?
10. What are the main rules to respect if you want a function to be convertible to a tf function?
11. When would you need to create a dynamic keras model? How do you do that? Why not make all your models dynamic?
12. Implement a custom layer that performs *Layer Normalisation*:
   * The `build()` method should define two trainable weights $\alpha$ & $\beta$, both of shape `input_shape[-1:]` & data type `tf.float32`. $\alpha$ should be initialised with 1s & $\beta$ with 0s.
   * The `call()` method should compute the mean $\mu$ & standard deviation $\sigma$ of each instance's features. For this, you can use `tf.nn.moments(inputs, axes = -1, keepdims = True)`, which returns the mean $\mu$ & the variance $\sigma^2$ of all instances (compute the square root of the variance to get the standard deviation). Then the function should compute & return $\alpha \otimes (X - \mu)/(\sigma + \varepsilon) + \beta$, where $\otimes$ represents itemwise multiplication ($*$) & $\varepsilon$ is a smoothing term (small constant to avoid division by zero, e.g., 0.001).
   * Ensure that your custom layer produces the same (or very nearly the same) output as the `keras.layers.LayerNormalization` layer.
13. Train a model using a custom training loop to tackle the fashion MNIST dataset.
   * Display the epoch, iteration, mean training loss, & mean accuracy over each epoch (updated at each iteration), as well as the validation loss & accuracy at the end of each epoch.
   * Try using a different optimiser with a different learning rate for the upper layers & the lower layers.

---

1. TensorFlow is a powerful library for large-scale machine learning or mathematical computation, that is also at the center of an ecosystem of libraries built for data validation, visualisation, preprocessing, all backed by a dedicated team of passionate developers & a large community contributing to improving it. It's core features include GPU support for multithreading, distributed computing (across multiple devices & servers), computation optimisation (for speed & memory usage), exportable computation graphs (train TensorFlow models in one environment (Python on Linux) & run it in another (Java on Android)), reverse-mode autodiff implementations, & optimisers (RMSProp & Nadam). Other popular deep learning libraries include pytorch & mxnet.
2. While TensorFlow can perform many of the same math operations as NumPy can, their differences lie mainly in the data types. TensorFlow is much stricter on performing operations with different data types. For example, in NumPy, you can add a float with an integer; but in TensorFlow, you cannot, because they are different data types. This is because when training large neural networks, type conversions can significantly increase training time, so TensorFlow will not automatically convert your data type. It will raise an exception if you execute an operation with incompatible types.

# 3.

In [2]:
import tensorflow as tf

tf.range(10)

<tf.Tensor: shape=(10,), dtype=int32, numpy=array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])>

In [4]:
import numpy as np

tf.constant(np.arange(10))

<tf.Tensor: shape=(10,), dtype=int32, numpy=array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])>

Yes, they seem the same.

4. TensorFlow supports several other data structures, beside regular tensors: *sparse tensors* (efficiently represent tensors containing mostly 0s), *tensor arrays* (lists of tensors of a fixed size, *all tensors must have the same shape & data type), *ragged tensors* (static lists of lists of tensors, where every tensor has the same shape & data type), *string tensors* (regular tensors of type `tf.string`), *sets* (regular or sparse tensors containing multiple sets of values {ex: `tf.constant([[1, 2], [3, 4]])` contains two sets {1, 2} & {3, 4}), & *queues* (store tensors across multiple steps).
5. If you create a custom loss function by writing a python function, the model containing your custom loss function can be saved & loaded, but you would have to manually set your loss function's hyperparameters every time you load your model. In other words, the hyperparameters of your custom loss function will not be saved. You can save the hyperparameters of your loss function along with your model by creating a subclass of the `keras.losses.Loss` class & implementing its `__init__()`, `call()`, & `get_config()` method. When you load your model this way, you just need to map the class name to the class, same as you would for a python function, but with a python function, you would have the specify the value for the hyperparameter.
6. Similar to loss functions, subclassing `keras.metrics.Metric` will save your metric's hyperparameters along with your model. Also, for metrics that cannot be averaged over batches, like precision, you must implement a streaming metric, which requires you to subclass `keras.metrics.Metric`. If you do not want to save your metrics hyperparameters along with the model, or your metric can be averaged over batches, then a simple python function will suffice.
7. You should create custom layers or custom blocks of layers if your model's architecture is very repetitive. For custom layers with no weights, you could wrap it in a `keras.layers.Lambda` layer, but with weights, you would subclass the `keras.layers.Layer` class. You should create a custom model if the model's architecture cannot be created given the tools of keras. For custom models, you should subclass the `keras.models.Model` class.
8. You might write a custom training loop if you want full control of of the training process, or if you just want to understand what's going on during model training, or if you want to use different optimisers for different parts of your neural network, like for the wide & deep example. Most of the time though, you want to avoid writing custom training loops because they are error-prone; you need to make sure a lot of thing are right for the loop to function properly.
9. 